# Guided Project: App Data Analysis

The goal of this project is to analyze data from the app store and discover what types of free apps might be most profitable to develop.

In this first cell, we define a new function called `explore_data()` that takes up to four arguments: the name of a dataset (which should be a list of lists), integers for the start and end that represent the size slice you want to look at (i.e. on which row you want to begin and end), and a Boolean called `rows_and_columns` which is `False` by default, but which will display the dataset's _total_ number of rows and columns if given the argument `True`.

So, for example, if you wanted to see the first two (real) rows of the ios dataset as well as count rows and columns, you would enter the argument `ios_data, 1, 3, True` for this function.

Note, also, that this assumes the dataset has no header row. If it does have one, you should enter the dataset as `dataset[1:]` or the function's count of rows will be off by one because it's counting the header row as a row of data.

In [1]:
def explore_data(dataset, start, end, rows_and_columns=False):
    dataset_slice = dataset[start:end]
    for row in dataset_slice:
        print(row)
        print('\n') #adds empty line after each row
        
    if rows_and_columns:
        print('Number of rows:', len(dataset))
        print('Number of columns:', len(dataset[0]))

In this cell, we import the data sets, open them, and convert them to lists of lists. Note that we need to specify `encoding="utf8"` as an argument in `open()` in order to get these files to read properly. 

In [2]:
from csv import reader

opened_ios = open('C:/users/Charlie/datasets/AppleStore.csv', encoding="utf8")
opened_android = open('C:/users/Charlie/datasets/googleplaystore.csv', encoding="utf8")

read_ios = reader(opened_ios)
read_android = reader(opened_android)

ios_data = list(read_ios)
android_data = list(read_android)

In [3]:
explore_data(ios_data[1:], 1, 2, True)
explore_data(android_data[1:], 1, 2, True)

['2', '281796108', 'Evernote - stay organized', '158578688', 'USD', '0', '161065', '26', '4', '3.5', '8.2.2', '4+', 'Productivity', '37', '5', '23', '1']


Number of rows: 7197
Number of columns: 17
['Coloring book moana', 'ART_AND_DESIGN', '3.9', '967', '14M', '500,000+', 'Free', '0', 'Everyone', 'Art & Design;Pretend Play', 'January 15, 2018', '2.0.0', '4.0.3 and up']


Number of rows: 10841
Number of columns: 13


That has given us a decent idea of how the data looks. Now, let's look at the header rows and assess which are likely to be the most useful.

In [4]:
print(ios_data[:1])
print(android_data[:1])

[['', 'id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num', 'vpp_lic']]
[['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']]


In `ios_data`, the most useful columns are probably `price` (index 5), `rating_count_tot` (index 6), `user_rating` (index 8), `content_rating` (index 11), and `prime_genre` (index 12).

In `android_data` the corresponding columns are `price` (index 7), `reviews` (index 3), `rating` (index 2), `Content Rating` (index 8) and `genres` (index 9).

In the Android dataset, `installs` might be useful as well, but it has no corresponding column in the iOS data.

## Cleaning the data

Now we're going to clean the data. From the dataset's source on Kaggle, we know that the Android data has an error in row 10472, so we're going to delete that from the dataset using `del[index]`.

So that we don't make a mistake, let's just print a few rows around the problem area to confirm we've got the right row. It's not clear whether the user's report of a mistake refers to the actual row 10472 or row 10472 of the dataset (which includes the header) so let's just double-check.

In [5]:
print(android_data[10471:10474])
print('\n')
print(android_data[10473])

[['Jazz Wi-Fi', 'COMMUNICATION', '3.4', '49', '4.0M', '10,000+', 'Free', '0', 'Everyone', 'Communication', 'February 10, 2017', '0.1', '2.3 and up'], ['Xposed Wi-Fi-Pwd', 'PERSONALIZATION', '3.5', '1042', '404k', '100,000+', 'Free', '0', 'Everyone', 'Personalization', 'August 5, 2014', '3.0.0', '4.0.3 and up'], ['Life Made WI-Fi Touchscreen Photo Frame', '1.9', '19', '3.0M', '1,000+', 'Free', '0', 'Everyone', '', 'February 11, 2018', '1.0.19', '4.0 and up']]


['Life Made WI-Fi Touchscreen Photo Frame', '1.9', '19', '3.0M', '1,000+', 'Free', '0', 'Everyone', '', 'February 11, 2018', '1.0.19', '4.0 and up']


This confirms that the issue is with `android_data[10473]`: the `Category` column was skipped in this row, and all the subsequent data points in this are thus not indexed correctly because they've been shifted one place earlier in the index. 

Let's go ahead and delete this row. Then, let's use `explore_data()` to check those rows, just to ensure everything looks correct.

In [6]:
del android_data[10473]
explore_data(android_data, 10471, 10475)

['Jazz Wi-Fi', 'COMMUNICATION', '3.4', '49', '4.0M', '10,000+', 'Free', '0', 'Everyone', 'Communication', 'February 10, 2017', '0.1', '2.3 and up']


['Xposed Wi-Fi-Pwd', 'PERSONALIZATION', '3.5', '1042', '404k', '100,000+', 'Free', '0', 'Everyone', 'Personalization', 'August 5, 2014', '3.0.0', '4.0.3 and up']


['osmino Wi-Fi: free WiFi', 'TOOLS', '4.2', '134203', '4.1M', '10,000,000+', 'Free', '0', 'Everyone', 'Tools', 'August 7, 2018', '6.06.14', '4.4 and up']


['Sat-Fi Voice', 'COMMUNICATION', '3.4', '37', '14M', '1,000+', 'Free', '0', 'Everyone', 'Communication', 'November 21, 2014', '2.2.1.5', '2.2 and up']




Now, let's clean the datasets of duplicate apps.

We're creating new empty lists for duplicate and unique apps in each dataset. Then, in separate `for` loops, we're looping through each row of the `ios_data` and `android_data` datasets to check whether the name already exists in the unique dataset list. If it does, it's added to the duplicate list. If it doesn't, it's added to the unique list.

Then we quickly explore the duplicate datasets to see how much we found.

In [7]:
duplicate_android = []
unique_android = []
duplicate_ios = []
unique_ios = []

for app in android_data:
    name = app[0]
    if name in unique_android:
        duplicate_android.append(name)
    else:
        unique_android.append(name)
        
for app in ios_data:
    name = app[2]
    if name in unique_ios:
        duplicate_ios.append(name)
    else:
        unique_ios.append(name)

print('Android duplicates:')        
explore_data(duplicate_android, 1, 10, True)
print('iOS duplicates:')
explore_data(duplicate_ios, 1, 10, True)

Android duplicates:
Box


Google My Business


ZOOM Cloud Meetings


join.me - Simple Meetings


Box


Zenefits


Google Ads


Google My Business


Slack


Number of rows: 1181
Number of columns: 28
iOS duplicates:
Mannequin Challenge


Number of rows: 2
Number of columns: 17


As we can see, the iOS dataset included just one duplicate entry, but the Android data included a lot (1,181!). We've now quarantined those duplicates, and created dupe-free datasets called `unique_ios` and `unique_android`. However, these new datasets aren't perfect, because in the code above, we removed the duplicate datasets randomly, storing whichever entry appeared first in the `'unique` dataset and removing the others. This probably doesn't matter for the iOS list, but for the Android list with so many duplicates, it might.

Instead of doing this, we probably want to store whichever entry has the highest number of user ratings, since that's likely to be the oldest and/or primary entry, and may give us the best measure of the app's popularity. Given that the Android dataset also includes number of installs (index 5), this might be a good measure as well, but this data is stored in an annoying string format (10M, 40K) so let's just stick with Ratings.

So let's do that cleaning again, but this time store the duplicate with the highest number of reviews in the `unique` dataset. First, let's get a measure of the different review counts we have for the Instagram app so that so that we can test to see if we've really gotten the highest one later when we make our dictionary.

In [8]:
for app in android_data[1:]:
    name = app[0]
    if name == 'Instagram':
        print(app[3])

66577313
66577446
66577313
66509917


Now we know that the HIGHEST review count for Instagram is **66,577,446**. So once we've created our dictionary of the highest number of reviews per app, that's the count we should get.

Let's create a dictionary called `reviews_max` and then look through `android_data`. If an app name appears in the data and its reviews count is less than what's in `reviews_max`, we want to replace the current definition for that app name in with the higher reviews count. If it isn't in the dictionary yet at all, then we want to add the app name and its reviews count to the dictionary.

In [9]:
reviews_max = {} 

for app in android_data[1:]:
    name = app[0]
    reviews = float(app[3])
    if name in reviews_max and reviews_max[name] < reviews:
        reviews_max[name] = reviews
    elif name not in reviews_max:
        reviews_max[name] = reviews
        
print(reviews_max['Instagram'])

print('Expected length:', len(android_data[1:]) - 1181)
print('Actual length:', len(reviews_max))


        

66577446.0
Expected length: 9659
Actual length: 9659


Great, it worked! Now we need to get this back into a list.

First, we'll creat a couple new empty lists. Then we'll loop through the original data set and pull app names and review numbers. For each app, we'll check if the listed review number is the same as what's in our `max_reviews` dictionary. If it is, and if the app's name hasn't been added yet, we'll add that app to the `android_clean` list. Then we'll add it to the `already_added` list just to have a record of what's been added. 

This should leave us with a list of each app that includes no duplicates and that has chosen the duplicate entry with the highest review count for each app. We'll use our `explore_data()` function to take a quick look at the new `android clean` list after building it to make sure everything looks right.


In [10]:
android_clean = []
already_added = []

for app in android_data[1:]:
    name = app[0]
    n_reviews = float(app[3])
    
    if (reviews_max[name] == n_reviews) and (name not in already_added):
        android_clean.append(app)
        already_added.append(name)
    
explore_data(android_clean, 0, 3, True)

['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']


['Sketch - Draw & Paint', 'ART_AND_DESIGN', '4.5', '215644', '25M', '50,000,000+', 'Free', '0', 'Teen', 'Art & Design', 'June 8, 2018', 'Varies with device', '4.2 and up']


Number of rows: 9659
Number of columns: 13


Now we want to try and remove apps that contain non-English names, since they're probably not aimed at an English speaking audience. We may not be able to easily detect other languages that use the English alphabet, but we can identify apps that use-non English alphabets in their names.

First, let's write a function and test it to see if it correctly identifies non-English app names.

In [11]:
def english_finder(string):
    for character in string:
        value = ord(character)
        if value > 127:
            return False
    return True

print(english_finder("Instagram"))
print(english_finder("爱奇艺PPS"))
print(english_finder("Instachat 😜"))

True
False
False


As we can see, this works, but it would also eliminate non-English names that use emoji. Let's rewrite it to only return false if there are more than 3 such characters in the app name.

In [12]:
def english_finder(string):
    n = 0
    for character in string:
        value = ord(character)
        if value > 127:
            n += 1
        if n >= 3:
            return False
    return True

print(english_finder("Instagram"))
print(english_finder("爱奇艺PPS"))
print(english_finder("Instachat 😜"))

True
False
True


Lovely! Let's now use that function to filter out the non-English apps in our `android_clean` and `ios_data` datasets:

In [13]:
android_eng = []
ios_eng = []
non_english = [] # just for checking to see if it's working correctly

for app in android_clean:
    name = app[0]
    is_english = english_finder(name)
    if is_english == True:
        android_eng.append(app)
    elif is_english == False:
        non_english.append(name)
        
for app in ios_data:
    name = app[2]
    is_english = english_finder(name)
    if is_english == True:
        ios_eng.append(app)
    elif is_english == False:
        non_english.append(name)
        
        
explore_data(android_eng, 0, 3, True)
explore_data(ios_eng, 0, 3, True)  
print(non_english[0:10])   

['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']


['Sketch - Draw & Paint', 'ART_AND_DESIGN', '4.5', '215644', '25M', '50,000,000+', 'Free', '0', 'Teen', 'Art & Design', 'June 8, 2018', 'Varies with device', '4.2 and up']


Number of rows: 9597
Number of columns: 13
['', 'id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num', 'vpp_lic']


['1', '281656475', 'PAC-MAN Premium', '100788224', 'USD', '3.99', '21292', '26', '4', '4.5', '6.3.5', '4+', 'Games', '38', '5', '10', '1']


['2', '281796108', 'Eve

Since in this exercise we're working for an app company that only makes free apps, we now want to reduce these datasets one more time to create lists with only the free apps.

In [14]:
android_free = []
ios_free = []
not_free = [] # just to test it's working correctly

for app in android_eng:
    price = app[6]
    if price == 'Free' or price == 'free' or price == 0:
        android_free.append(app)
    else:
        not_free.append(app)
        
        # I did the above loop the hard way, using index 7 (numeric price) would have been simpler...
        
for app in ios_eng[1:]:
    price = float(app[5])
    if price == 0:
        ios_free.append(app)
    else:
        not_free.append(app)
        
explore_data(android_free, 0, 3, True)
explore_data(ios_free, 0, 3, True)
explore_data(not_free, 0, 3, True)

['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']


['Sketch - Draw & Paint', 'ART_AND_DESIGN', '4.5', '215644', '25M', '50,000,000+', 'Free', '0', 'Teen', 'Art & Design', 'June 8, 2018', 'Varies with device', '4.2 and up']


Number of rows: 8847
Number of columns: 13
['2', '281796108', 'Evernote - stay organized', '158578688', 'USD', '0', '161065', '26', '4', '3.5', '8.2.2', '4+', 'Productivity', '37', '5', '23', '1']


['3', '281940292', 'WeatherBug - Local Weather, Radar, Maps, Alerts', '100524032', 'USD', '0', '188583', '2822', '3.5', '4.5', '5.0.0', '4+', 'Weather', '37', '5', '3', '1']


['4', '282614216', 'eBay: Best App to Buy, Sell, Save! Online Shop

## Data Analysis

Now that we've got some relatively clean lists of data in `android_free` and `ios_free`, it's time to start actually analyzing stuff. The first thing we'll probably want to do is look at the **genres** (index 9 for android and 12 for ios). Category (index 1) might also be useful for Android.

Annoyingly, because there are a lot of genres, we'll want to create sorted frequency tables. There is a `sorted()` function in Python, but it doesn't work well with dictionaries, so we'll need to create dictionaries with the genre as the key, the number of apps in that genre as the dictionary value, and then turn that dictionany into a list of tuples. We'll use the provided `display_table()` function to do this and to display the sorted tables, but first we need to create a `freq_table` function that can take our list of lists and turn it into a frequency table.

In [15]:
def freq_table(list, index):
    frequency = {}
    for app in list:
        genre = app[index]
        if genre in frequency:
            frequency[genre] += 1
        else:
            frequency[genre] = 1
    return frequency

print(freq_table(ios_free, 12))

{'Productivity': 56, 'Weather': 28, 'Shopping': 83, 'Reference': 17, 'Finance': 35, 'Music': 66, 'Utilities': 79, 'Travel': 40, 'Social Networking': 106, 'Sports': 69, 'Health & Fitness': 65, 'Games': 1866, 'Food & Drink': 26, 'News': 43, 'Book': 12, 'Photo & Video': 160, 'Entertainment': 251, 'Business': 17, 'Lifestyle': 50, 'Education': 118, 'Navigation': 6, 'Medical': 6, 'Catalogs': 4}


That seems to be working, so let's try to combine it with the pre-written `display_table()` function provided by Dataquest to see if we can get a properly sorted table.

In [16]:
def display_table(dataset, index):
    table = freq_table(dataset, index)
    table_display = []
    for key in table:
        key_val_as_tuple = (table[key], key)
        table_display.append(key_val_as_tuple)

    table_sorted = sorted(table_display, reverse = True)
    for entry in table_sorted:
        print(entry[1], ':', entry[0])

print('iOS table:')        
print(display_table(ios_free, 12))
print('\n')
print('Android table Category:')
print(display_table(android_free, 1))
print('\n')
print('Android table Genre:')
print(display_table(android_free, 9))

iOS table:
Games : 1866
Entertainment : 251
Photo & Video : 160
Education : 118
Social Networking : 106
Shopping : 83
Utilities : 79
Sports : 69
Music : 66
Health & Fitness : 65
Productivity : 56
Lifestyle : 50
News : 43
Travel : 40
Finance : 35
Weather : 28
Food & Drink : 26
Reference : 17
Business : 17
Book : 12
Navigation : 6
Medical : 6
Catalogs : 4
None


Android table Category:
FAMILY : 1675
GAME : 858
TOOLS : 748
BUSINESS : 407
PRODUCTIVITY : 345
LIFESTYLE : 344
FINANCE : 328
MEDICAL : 313
SPORTS : 300
PERSONALIZATION : 294
COMMUNICATION : 286
HEALTH_AND_FITNESS : 273
PHOTOGRAPHY : 261
NEWS_AND_MAGAZINES : 248
SOCIAL : 236
TRAVEL_AND_LOCAL : 207
SHOPPING : 199
BOOKS_AND_REFERENCE : 189
DATING : 165
VIDEO_PLAYERS : 159
MAPS_AND_NAVIGATION : 123
FOOD_AND_DRINK : 110
EDUCATION : 103
ENTERTAINMENT : 85
LIBRARIES_AND_DEMO : 83
AUTO_AND_VEHICLES : 82
HOUSE_AND_HOME : 71
WEATHER : 70
EVENTS : 63
PARENTING : 58
ART_AND_DESIGN : 57
COMICS : 54
BEAUTY : 53
None


Android table Genre:
Tool

Looking at these tables, we can see some obvious trends. The iOS market is saturated with free games, with relatively few reference, book, and medical apps. The Android market looks similar on the top end (games) but there is significantly more in the way of books and reference apps.

Now let's figure out which genres of apps have the most users. We can do this with Installs (index 5) for Android and rating count (index 6) for iOS. Let's start with iOS.

In [17]:
ios_genres = {}
ios_genres = freq_table(ios_free, 12)

for genre in ios_genres:   #for each genre, set total to zero and length to zero
    total = 0
    len_genre = 0
    
    for app in ios_free:       #then for each app in the dataset...
        genre_app = app[12]    
        if genre_app == genre:   # see if the genre there matches the genre from our freq table, and if it does...
            ratings = app[6]      # define which index number is the ratings count
            total += float(ratings)   # add that number to our total
            len_genre += 1           #and add 1 to the genre length, ie, total number of genres
            
    average = total / len_genre       # find average number of ratings by genre
    rounded_avg = round(average, 1)
    print(genre, ':', rounded_avg)
    

Productivity : 21028.4
Weather : 52279.9
Shopping : 27230.7
Reference : 79350.5
Finance : 32367.0
Music : 57326.5
Utilities : 19156.5
Travel : 28243.8
Social Networking : 71548.3
Sports : 23008.9
Health & Fitness : 23298.0
Games : 22886.4
Food & Drink : 33333.9
News : 21248.0
Book : 46384.9
Photo & Video : 28441.5
Entertainment : 14195.4
Business : 7491.1
Lifestyle : 16815.5
Education : 7004.0
Navigation : 86090.3
Medical : 612.0
Catalogs : 4004.0


And now Android, which is slightly trickier because we need to get rid of the commas and pluses for the install counts to get flat numbers.


In [18]:
android_genres = {}
android_genres = freq_table(android_free, 1)

for genre in android_genres:   #for each genre, set total to zero and length to zero
    total = 0
    len_genre = 0
    
    for app in android_free:       #then for each app in the dataset...
        genre_app = app[1]    
        if genre_app == genre:   # see if the genre there matches the genre from our freq table, and if it does...
            installs = app[5]      # define which index number is the ratings count
            installs = installs.replace(',','') #replace commas with nothing
            installs = installs.replace('+','') #replace pluses with nothing
            total += float(installs)   # add that number to our total
            len_genre += 1           #and add 1 to the genre length, ie, total number of genres
            
    average = total / len_genre       # find average number of ratings by genre
    rounded_avg = round(average, 1)
    print(genre, ':', rounded_avg)

ART_AND_DESIGN : 1986335.1
AUTO_AND_VEHICLES : 647317.8
BEAUTY : 513151.9
BOOKS_AND_REFERENCE : 8814199.8
BUSINESS : 1712290.1
COMICS : 832613.9
COMMUNICATION : 38590581.1
DATING : 854028.8
EDUCATION : 1833495.1
ENTERTAINMENT : 11640705.9
EVENTS : 253542.2
FINANCE : 1387692.5
FOOD_AND_DRINK : 1924897.7
HEALTH_AND_FITNESS : 4188822.0
HOUSE_AND_HOME : 1360598.0
LIBRARIES_AND_DEMO : 638503.7
LIFESTYLE : 1446158.2
GAME : 15544014.5
FAMILY : 3697848.2
MEDICAL : 120550.6
SOCIAL : 23253652.1
SHOPPING : 7036877.3
PHOTOGRAPHY : 17840110.4
SPORTS : 3650602.3
TRAVEL_AND_LOCAL : 13984077.7
TOOLS : 10830252.0
PERSONALIZATION : 5201482.6
PRODUCTIVITY : 16787331.3
PARENTING : 542603.6
WEATHER : 5145550.3
VIDEO_PLAYERS : 24727872.5
NEWS_AND_MAGAZINES : 9549178.5
MAPS_AND_NAVIGATION : 4049274.6


So, looking at these results and comparing with the counts, some recommendations for apps that tend to have high install numbers but comparatively low market saturation. 

iOS: Reference, Weather, Utilities
Android: Reference

However, this is a bit difficult to look at because these tables aren't actually in order. Let's see if we can re-write this to generate a sorted frquency table?

In [19]:
android_genres = {}
android_genres = freq_table(android_free, 1)
from operator import itemgetter

for genre in android_genres:   #for each genre, set total to zero and length to zero
    total = 0
    len_genre = 0
    
    for app in android_free:       #then for each app in the dataset...
        genre_app = app[1]    
        if genre_app == genre:   # see if the genre there matches the genre from our freq table, and if it does...
            installs = app[5]      # define which index number is the ratings count
            installs = installs.replace(',','') #replace commas with nothing
            installs = installs.replace('+','') #replace pluses with nothing
            total += float(installs)   # add that number to our total
            len_genre += 1           #and add 1 to the genre length, ie, total number of genres
            
    average = total / len_genre       # find average number of ratings by genre
    rounded_avg = round(average, 0)
    unsorted_table = {}
    unsorted_table[genre] = rounded_avg   # putting the values into a new dictionary
    table_display = []
    for key in unsorted_table:
        key_val_as_tuple = (unsorted_table[key], key)
        table_display.append(key_val_as_tuple)

    table_sorted = sorted(table_display, key=itemgetter(0), reverse=True)  # HOW CAN WE SORT BY SECOND VALUE IN TUPLE?
    for entry in table_sorted:
        print(entry[0], ':', entry[1])
    

1986335.0 : ART_AND_DESIGN
647318.0 : AUTO_AND_VEHICLES
513152.0 : BEAUTY
8814200.0 : BOOKS_AND_REFERENCE
1712290.0 : BUSINESS
832614.0 : COMICS
38590581.0 : COMMUNICATION
854029.0 : DATING
1833495.0 : EDUCATION
11640706.0 : ENTERTAINMENT
253542.0 : EVENTS
1387692.0 : FINANCE
1924898.0 : FOOD_AND_DRINK
4188822.0 : HEALTH_AND_FITNESS
1360598.0 : HOUSE_AND_HOME
638504.0 : LIBRARIES_AND_DEMO
1446158.0 : LIFESTYLE
15544015.0 : GAME
3697848.0 : FAMILY
120551.0 : MEDICAL
23253652.0 : SOCIAL
7036877.0 : SHOPPING
17840110.0 : PHOTOGRAPHY
3650602.0 : SPORTS
13984078.0 : TRAVEL_AND_LOCAL
10830252.0 : TOOLS
5201483.0 : PERSONALIZATION
16787331.0 : PRODUCTIVITY
542604.0 : PARENTING
5145550.0 : WEATHER
24727872.0 : VIDEO_PLAYERS
9549178.0 : NEWS_AND_MAGAZINES
4049275.0 : MAPS_AND_NAVIGATION


OK, that didn't work. However, it wasn't strictly necessary, and simply changing the order they're displayed in allows us to easily copy-paste into a spreadsheet and sort that way. That reveals that the top apps on average are news and magazines, books and reference, dating, and comics. Assuming we're looking for non-crowded, high-download apps, **Comics** and **Books and Reference** seem like the best bets, and would probably be easy apps to build to boot (the challenge would be licensing content, probably).